In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Thu Oct 31 08:44:36 PDT 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.971
BogoMIPS:              5851.97
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables
`%env` variables are good for passing to bash cells

In [2]:
# Set workding directory
%env wd=/home/sam/analyses/20191105_swoose_pgen_v074_renaming
wd="/home/sam/analyses/20191105_swoose_pgen_v074_renaming"

%env rsync_owl=owl:/volume1/web/halfshell/genomic-databank/
%env files_list=Panopea-generosa-vv0.74.a4.CDS.gff3 Panopea-generosa-vv0.74.a4.exon.gff3 Panopea-generosa-vv0.74.a4.gene.gff3 Panopea-generosa-vv0.74.a4.mRNA.gff3 Panopea-generosa-vv0.74.a4.repeat_region.gff3 Panopea-generosa-vv0.74.a4.rRNA.gff3 Panopea-generosa-vv0.74.a4.tRNA.gff3 Pgenerosa_v074.fa Pgenerosa_v074.fa.fai Pgenerosa_v074.CpG.gff Panopea-generosa-vv0.74.a4.intergenic.bed  Panopea-generosa-vv0.74.a4.introns.bed Panopea-generosa-vv0.74.a4.repeats.DNA.gff3 Panopea-generosa-vv0.74.a4.repeats.LTR.gff3 Panopea-generosa-vv0.74.a4.repeats.SINE.gff3 Panopea-generosa-vv0.74.a4.repeats.Unknown.gff3 Panopea-generosa-vv0.74.a4.repeats.LINE.gff3 Panopea-generosa-vv0.74.a4.repeats.RC.gff3 Panopea-generosa-vv0.74.a4.repeats.Simple_repeat.gff3
%env wget_command=--directory-prefix=${wd} --quiety --no-directories --no-check-certificate  https://owl.fish.washington.edu/halfshell/genomic-databank/

%env new_prefix=Panopea-generosa-v1.0
%env fa_prefix=Pgenerosa_v074
%env gff_prefix=Panopea-generosa-vv0.74

env: wd=/home/sam/analyses/20191031_pgen_v074_stringtie_BAM_splitting
env: rsync_gannet=gannet:/volume2/web/Atumefaciens/20190723_stringtie_pgen_v074/
env: wget_bam=--quiet --no-check-certificate https://gannet.fish.washington.edu/Atumefaciens/20190723_stringtie_pgen_v074/20190723_sorted.merged.bam
env: original_bam=20190723_sorted.merged.bam
env: reassembled_bam=20190723_sorted.merged.reassembled.bam


#### Create necessary directories

In [3]:
%%bash
mkdir --parents ${wd}

In [4]:
cd {wd}

/home/sam/analyses/20191031_pgen_v074_stringtie_BAM_splitting


#### Download Pgen_v074 files


In [ ]:
%%bash
# Create array of files from list
mapfile -t files_array < <(echo "${files_list}" | tr " " "\n")

for file in "${files_array[@]}"
do
  rsync \
  --archive \
  --progress \
  --verbose \
  "${rsync_owl}${file}" \
  .
done

ls -lh

#### If need to download via wget, uncomment lines in the cell below

In [5]:
# %%bash
# time \
# wget "${wget_gffs}"
# wget "${wget_fasta}"
# ls -lh ${wd}

In [7]:
%%bash
ls -lh

total 73G
-rw-rw-r-- 1 sam users 73G Aug 29 10:54 20190723_sorted.merged.bam


In [ ]:
%%bash

mapfile -t files_array < <(echo "${files_list}" | tr " " "\n")

# Array of old scaffold names
# Uses first column of FastA index file to get scaffold names
mapfile -t orig_scaffold_names < <(awk '{print $1}' Pgenerosa_v074.fa.fai)

# Array of new scaffold names
mapfile -t new_scaffold_names < <(for number in {01..18}; do echo "Scaffold_${number}"; done)


for file in "${files_array[@]}"
do
  # Set new filename, depending if FastA or GFF
  name_check=$("${file%%.*}")
  if [ "${name_check}" = "${fa_prefix}" ]
  then
    new_name=${file/$name_check/$fa_prefix}
  else
    new_name=${file/$name_check/$gff_prefix}
  fi
  # sed substitution
  # creates sed script to find original scaffold names and replace them with new scafold names
  # and passes to sed via stdin
  for index in "${!orig_scaffold_names[@]}"
  do
    printf 's/%s/%s/'\n "${orig_scaffold_names[index]}" "${new_scaffold_names[index]}"
  done \
  | sed --file- "${file}" \
  >> "${new_name}"
done